In [1]:
import json

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [3]:
import import_ipynb
from aux import utils
from aux import nlp
from aux import relation_extraction
from aux import defs
import preparation
import rule_base

importing Jupyter notebook from /Users/YK/mt/project/aux/utils.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/nlp.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/relation_extraction.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/defs.ipynb
importing Jupyter notebook from preparation.ipynb
importing Jupyter notebook from rule_base.ipynb


In [4]:
class RuleExplanation06(rule_base.Rule):
    name = "explanation_06"
    relation_type = "Explanation"
    reasons = {
        "COMMON_PATTERN_-_ELABORATION->>BACKGROUND": 
            defs.Reason(
                1,
                "Common pattern ( -Elaboration->>Background)."
            ),
        "COMMON_PATTERN_-_JOINT->>BACKGROUND": 
            defs.Reason(
                1,
                "Common pattern ( -Joint->>Background)."
            )
    }
    
    def generate_statement(self, text, relation, verbose=False, **kwargs):
        assert(relation is not None and relation.type == "Explanation")
        info = preparation.Preprocessor.prepare_extended_info(text, relation, verbose)
        if info is None:
            utils.print_if_verbose("Extended info preparation wasn't successful.", verbose)
            return None

        if info.satellite_info.relation is None:
            return None
        
        utils.print_if_verbose(
                "Nucleus relation type: "
                f"'{utils.get_relation_type(info.nucleus_info.relation)}'.",
                verbose
            )
        utils.print_if_verbose(
            "Satellite relation type: "
            f"'{utils.get_relation_type(info.satellite_info.relation)}'.",
            verbose
        )
            
        reason = None
        
        assert info.satellite_info.relation is not None
        if info.nucleus_info.relation is None:
            if (
                info.satellite_info.relation.type == "Elaboration"
                    and utils.has_nested_background(info.satellite_info.relation)
            ):
                reason = self.reasons["COMMON_PATTERN_-_ELABORATION->>BACKGROUND"]
            elif (
                info.satellite_info.relation.type == "Joint"
                    and utils.has_nested_background(info.satellite_info.relation)
            ):
                reason = self.reasons["COMMON_PATTERN_-_JOINT->>BACKGROUND"]

        if reason is not None:
            utils.print_if_verbose(reason.explanation, verbose)
            final_nucleus_text, final_sn_text = self._finalise_statement_parts(
                info.nucleus_preparation_result.prepared_text,
                info.satellite_preparation_result.prepared_text,
                verbose,
                add_punctuation=True
            )
            return self._generate_statement(
                final_nucleus_text, 
                " Also, ", 
                final_sn_text, 
                relation, 
                info.nucleus_proximity,
                self.name,
                reason,
                verbose
            )
        else:
            return None

In [5]:
if __name__ == "__main__" and "__file__" not in globals():
    rule = RuleExplanation06()

    with open("../parsed/race/train/middle/2708.txt.tree", "rt") as f:
        tree_text = f.read()

    text, relations = relation_extraction.read_relations(
        tree_text.replace("<s>", "").replace("<P>", "")
    )

    expl = relations["Explanation"][1]
    print(text[expl.left.start:expl.right.end])

    statement =rule.generate_statement(text, expl, verbose=True)
    print("\nRESULT:")
    if statement is not None:
        print(json.dumps(statement._asdict(), indent=2))

You can give your parents a happy day with a card or a joke .  It 's also lovely when a kid cleans up his or her room without being asked .  And if you try not fight with your brothers or sisters , your parents will be so happy .  Do your best at whatever you do .  
Nucleus is on the left.
Nucleus's depth <= 100.
Nucleus is flat.
Will use the whole segment.
Satellite's nucleus is on the left.
Nuclei proximity is NucleusProximity.NEAR
Satellite's (left) nucleus contains '.', '!', '?', or ';'.
Will use the whole segment.
Text extracted from the satellite:
It's also lovely when a kid cleans up his or her room without being asked.  And if you try not fight with your brothers or sisters, your parents will be so happy.  Do your best at whatever you do.
Nucleus relation type: '-'.
Satellite relation type: 'Joint'.
Common pattern ( -Joint->>Background).
Taking the last sentence and resolving pronouns:
You can give your parents a happy day with a card or a joke. 
---> 
You can give your parents